In [12]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import requests 
import re
from datetime import datetime
import csv
import pdb
import pymongo
from pymongo import MongoClient

In [3]:
browser = webdriver.PhantomJS(executable_path='./phantomjs') 
#browser.get('http://search.appledaily.com.tw/appledaily/search')
browser.get('https://tw.news.appledaily.com/search/')
browser

<selenium.webdriver.phantomjs.webdriver.WebDriver (session="d18c7620-e3a2-11e7-8d15-c1dc862bf2da")>

In [4]:
f = open('./data_2017.csv', 'w')
headers = ['id', 'title', 'datetime', 'content', 'hash', 'media', 'category', 'url', 'reporter', 'location']
writer = csv.DictWriter(f, fieldnames=headers)
writer.writeheader()

In [5]:
browser.find_element_by_id('searchFieldAnd').send_keys('政治')
time.sleep(2)
browser.find_element_by_id('start_date').clear()
time.sleep(2)
browser.find_element_by_id('start_date').send_keys('2017-01-01')
browser.find_element_by_id('end_date').clear()
browser.find_element_by_id('end_date').send_keys('2017-12-17')
browser.find_element_by_id('searchFieldAnd').send_keys(Keys.ENTER)
time.sleep(2)

In [6]:
pre_soup = bs(browser.page_source, 'lxml')
page_temp = pre_soup.select('#pageNumber a')
page_count = []
for pt in page_temp:
    page_count.append(pt.text)
time.sleep(2)
page_count = int(page_count[-3])

In [7]:
page_count=2
count = 0
list_all = [] 
browser.implicitly_wait(5)
while count < page_count :
#     starttime = time.clock() 
#     endtime = time.clock() 
#     print (endtime-starttime) #程序实体位置
    soup = bs(browser.page_source, 'lxml')
    select_portion = soup.select('#searchBox h2 > a')
    try:
        for s  in select_portion:   
            article = {}
            ursplit = s.attrs['href'].split("/")
            soup = bs(requests.get(s.attrs['href']).text,'html5lib')
            t= soup.select("div.ndArticle_margin > p")[0].text.strip().replace('\xa0', '').replace('\u3000', '  ')

            article["id"]=(ursplit[-1]) 
            article["title"] = s.text.replace('\u200b', '  ').replace('\u3000', '  ')
            article["datetime"]=datetime.strptime(soup.select("div.ndArticle_creat")[0].text.strip().replace('出版時間：', '')[:10],"%Y/%m/%d").strftime("%Y-%m-%d")
            article["content"]= t
            article["hash"]=hash(requests.get(s.attrs['href']).text)
            article["media"]="appledaily"
            article["category"]="politics"
            article["url"] = s.attrs['href']
            article["reporter"] = None
            article["location"] = None

            print(article)
            list_all.append(article)
            writer.writerow(article)
            f.flush()
            

        count += 1
        print("page no.:%s"%(count))
    except:pass  
    try:
        if count < page_count:
            browser.find_element_by_css_selector('img[alt="下一頁"]').click()
            
    except:
        print("!!!!!!")
        continue
        

{'id': '1260974', 'title': '反空污大遊行  成藍綠陣營角力場', 'datetime': '2017-12-17', 'content': '（更新：新增各地情況）國人關心空污議題，使反空污場子變成政治表態競技場！台中市長林佳龍、彰化縣長魏明谷、嘉義市長涂醒哲、南投縣長林明溱和雲林縣副縣長丁彥哲今天參加反空污大遊行，國民黨也由副主席郝龍斌帶領表態參選台中市長的立委盧秀燕、江啟臣等人參與，連有意參選台北市長的立委蔣萬安也到場，呈現「藍綠同心反空污同場遊行」的難得畫面。林佳龍表示，明年起更將搭配許可證展延時機，對總排放量前50大汙染源，強制要求業者採行汙染防制設備「最佳可行控制技術」（BACT），經審查後將記載於許可證內容核發，以實際行動控管業者落實空汙減排。江啟臣晚間致詞希望中央體會人民的痛苦，嚴正考慮將行政立法南遷中部，讓台北的中央政府官員體會中南會呼吸到的空氣與感受。盧秀燕則說，她今年二月已經參加過反空污遊行，但二月全民上街頭政府不聽，今天再上街頭，希望政府檢討能源政策，還我們清淨空氣。林明溱指出，南投縣好山好水，但台中火力發電廠，造成南投縣空氣品質非常差，他特別站出來，呼籲火力發電廠一定要減量。由於火力發電是空污主要來源之一，有人主張增加核能發電比例，但涂醒哲堅決反對，「應用替代能源取代污染能源，而不是走回頭路恢復核能！」雲林縣長李進勇原本跟主辦單位報名參加，今天因故未出席，改由丁彥哲代表參加遊行。丁彥哲指出，雲林縣冬季空氣品質不佳，該縣除車輛管制、街道洗掃、揚塵防治等措施外，也針對六輕離島工業區祭出全國最嚴格的空污管制規範，多管齊下使今年PM2.5年濃度較往年改善。南部反空污大聯盟今天也在高雄發起遊行，提出反空污連署承諾書，要求政治人物簽署。有意參選高雄市長的民進黨5位參選人，僅有立委林岱樺到場上台說明，有條件簽署反空污連署承諾書，此外立委管碧玲雖簽署承諾書，但未到場，而立委劉世芳則參與部分遊行、未簽署承諾書。立委陳其邁派代表參與遊行，並傳真回覆連署書，二項支持，三項有條件支持。立委趙天麟則派員到場關心，但未參與遊行也未簽承諾書。國民黨由黨主席吳敦義領軍，多位高雄、台南市市議員陪同。（地方中心／台中-高雄連線報導）出版時間:1313更新時間:2302', 'hash': 7063707635991185204, 'media': '

{'id': '1260824', 'title': '【罷昌未過】同意票遠高於不同意仍難罷免  學者這樣分析', 'datetime': '2017-12-17', 'content': '（更新：增加網友意見）時代力量立委黃國昌罷免案投票結果，有4.8萬當地選民投下同意票，2.1萬人投反對票，雖然同意是不同意票的兩倍，但因未達所有選民四分之一的門檻6.3萬票，罷免未通過。學者分析罷免理由不太正確，且民眾對罷免投票不積極，是罷免案難過關的主因。東吳大學政治系教授黃秀端表示，黃國昌罷免案理由本身就有點奇怪，先是因支持同性婚姻，後又延伸為他不在地方做服務，但立委的工作本來就是在立法院，地方服務是議員的工作，罷免理由不太正確；再者，彈劾跟罷免是不一樣的，彈劾是要做錯事，而罷免是政治性，只要不同意見即可提罷免，但黃國昌在立委評鑑算是表現很好，因此為了求穩定，罷免案的門檻提高與否是可以再討論。元智大學人文社會學院院長邱昌泰表示，民眾對罷免案的投票行為沒那麼積極，因此成功機率不大。對於贊成罷免的票數達4萬多票，邱昌泰認為，主要是國民黨動員系統發動，但這不算是成功的動員，藍軍的氣勢並沒有發揮出來；不過，黃國昌罷免案雖沒成功，但黃下次選舉恐面臨選民分裂，難以再拿到第一次選舉的高票數。中山大學政治研究所教授廖達琪說，投票結果對時力的基層動員是很大警訊，儘管罷免未過，但對時力「不算是好消息」，顯見在地方紮根困難度不低。廖也認為，罷免案沒過且罷免理由包括宗教力量與政黨算計，不至於對立委問政造成「寒蟬效應」。\u200b實踐大學博雅學部副教授賴岳謙表示，黃國昌長期以來的政治信念是「罷免或公投應該簡單多數就通過」，本次投票同意罷免選民比反對者多了兩萬多票，黃應該依其政治信念自行請辭立委。網友Cheng Taw Wu說，支持國昌的只剩2.1萬票，不同意他的却有4.8萬，現在不辭，那他下次也不要用選了！網友Peter King說，這次罷免黃國昌不成功，大家不要灰心，下次罷免國民黨的35席立委，保證個都成功。（政治中心／台北報導）發稿時間：01：30更新時間：09 : 06', 'hash': -1755333406890626701, 'media': 'appledaily', 'category': 'politics', 'url': 'https://tw.news.appledail

{'id': '', 'title': '催出4.8萬罷免票 安盟：持續監督', 'datetime': '2017-12-17', 'content': '時代力量立委黃國昌罷免案，昨雖未過關，但4萬8693同意票足讓黃國昌敲響警鐘，提案罷免的安定力量聯盟未來動向備受矚目，安定力量主席孫繼正說，未來會成為監督民代的團體，替民眾提供發聲平台，他個人沒有參選計劃，以家庭為重。', 'hash': 5421426221421557416, 'media': 'appledaily', 'category': 'politics', 'url': 'http://tw.news.appledaily.com/headline/daily/20171217/37876637/', 'reporter': None, 'location': None}
{'id': '', 'title': '姚文智今宣布選北市 綠有雜音', 'datetime': '2017-12-17', 'content': '【綜合報導】民進黨立委姚文智今將宣布參選台北市長，闡述治理藍圖。綠營內部卻傳出，擬推行政院長賴清德參選北市長，企圖以「賴神戰柯P」，將國民黨邊緣化，有利整體選情。行政院發言人徐國勇昨說，賴揆很努力推動政策，沒有想過這些事。姚文智說，任何人要選，他都歡迎。民進黨說，尊重選對會的討論跟決定。', 'hash': -4411389673866605188, 'media': 'appledaily', 'category': 'politics', 'url': 'http://tw.news.appledaily.com/headline/daily/20171217/37876648/', 'reporter': None, 'location': None}
{'id': '', 'title': '中研院2學者 申請入港遭拒', 'datetime': '2017-12-17', 'content': '【綜合報導】繼文化總會副秘書長張鐵志日前遭香港拒絕入境後，昨又傳出有兩名中研院學者欲赴港參加學術研討會，但申請港簽遭拒。學者批評，香港政府拒絕台人入境的「黑名單」已從社運界擴大到學術界，代表香港政治自由正快速倒退。我政府表達遺憾與不滿，盼港方勿干擾台港民間交流。港府入境處則回

In [42]:
import os

files = [file for file in os.listdir() if file.endswith(".csv") and file.startswith("applenews")]

In [17]:
import csv
import json
for file in files:
    with open(file, "r") as f:
        news = list(csv.DictReader(f))
    n = [dict(i) for i in news]
    for i in n:
        if i["id"] == "":
            i["id"] = None
        if i["reporter"] == "":
            i["reporter"] = None
        if i["location"] == "":
            i["location"] = None
    with open("change_" + file, "w") as f:
        json.dumps(n, ensure_ascii=False).encode('utf8')
        f.write()

In [36]:
n = [dict(i) for i in news]

In [37]:
for i in n:
    if i["id"] == "":
        i["id"] = None
    if i["reporter"] == "":
        i["reporter"] = None
    if i["location"] == "":
        i["location"] = None

In [38]:
n

[{'category': 'politics',
  'content': '一名14歲女童本月23日在金鐘原佔領區一處名為「連儂牆」的牆壁上用粉筆畫花，遭警方以涉嫌刑事毀壞拘捕。其後即使女童父親表明有能力照顧女兒，但法庭仍指父親有聽障無能力照顧女童，將女童判入屯門兒童及青少年院3星期。香港民主黨質疑警方以極嚴厲手段對未成年少女，是政治打壓，企圖製造白色恐怖，要求警方交代。香港學民思潮也發起聯署聲援該名14歲女童，表達對警方向法庭申請兒童保護令的不滿，學民建立的facebook活動聯署專頁，截至下午5時，已有逾4.5萬人表明參與聯署。被警方扣留長達17小時的女童，今下午由少女的代表律師替她申請保釋，在控方不反對下，高等法院法官批准申請，並下令即日釋放少女。少女的父親在庭外指，對女兒獲准保釋感到開心。法官湯寶臣指，發出保護令是以女童的利益及有否受不良影響作依歸，解釋本案並非刑事案。他又解釋，保釋申請本定於周五處理，但因得悉律政司一方並不反對，故即時開庭緊急處理。（國際中心／綜合外電報導）【學聯聯署聲名全文】上星期，一名14歲的中三女童於金鐘政府總部「連儂牆」上以粉筆繪花，被十數名警察包圍，並以涉嫌刑事毀壞拘捕，在拘留近20小時後，警方於昨天在少年法庭向該名少女申請保護令，法庭終裁定少女需由社署接管並送入屯門兒童及青少年院三星期時間，期間將無法如常上學。我們在此強烈遣責警向該名女童申請保護令，認為警方根本不能證明該女童的不受控制程度，有機會引致她本人或其他人再度受到傷害，亦質疑法庭判決是否合理和公正，難道在牆上以粉筆繪花便能證明女童會再度傷人？即 使女童之父親已表明有能力照顧其女兒，但法庭仍將女童判入兒童及青少年院，而法庭亦指該父親因患聽障而沒有能力照顧女兒，實在沒有合理根據，我們在此表示 極度無奈及憤怒，更讓我們一度懷疑法院判決違反《兒童權利公約》第9條：「確保不違背兒童父母的意願使兒童與父母分離，除非經過法院判定分離符合兒童的最 大利益，而且有其必要」，而法官亦務必交待為何在未來跟父母分離二十一天，居然能給予女童的最大利益和保障。對此不合理之裁決，我們希望 警方清晰向公眾交待，為何在除夕臨近之際，突然需向女童申請保護令， 是否以「保護」之名，卻是恐嚇為實，意圖打擊走上街頭的年輕學子，而法庭亦本應為小市民作出公正判決，方能彰顯社會公義，以保障人民權益，貫徹「以法達 

In [ ]:
# page_count=10
# count = 0
# list_all = [] 
# browser.implicitly_wait(5)
# while count < page_count :
# #     starttime = time.clock() 
# #     endtime = time.clock() 
# #     print (endtime-starttime) #程序实体位置
#     soup = bs(browser.page_source, 'lxml')
#     select_portion = soup.select('#searchBox h2 > a')
    
#     for s in select_portion: 
#         try:
#             article = {}
#             ursplit = s.attrs['href'].split("/")
#             soup = bs(requests.get(s.attrs['href']).text,'html5lib')
#             t= soup.select("div.ndArticle_margin > p")[0].text.strip().replace('\xa0', '').replace('\u3000', '  ')

#             article["id"]=(ursplit[-1]) 
#             article["title"] = s.text.replace('\u200b', '  ').replace('\u3000', '  ')
#             article["datetime"]=datetime.strptime(soup.select("div.ndArticle_creat")[0].text.strip().replace('出版時間：', '')[:10],"%Y/%m/%d").strftime("%Y-%m-%d")
#             article["content"]= t
#             article["hash"]=hash(requests.get(s.attrs['href']).text)
#             article["media"]="appledaily"
#             article["category"]="politics"
#             article["url"] = s.attrs['href']
#             article["reporter"] =""
#             article["location"] =""

#             print(article)
#             list_all.append(article)
#         except:
#             pass

#         count += 1
# #         print("page no.:%s"%(count))
    
#     try:
#         if count < page_count:
#             browser.find_element_by_css_selector('img[alt="下一頁"]').click()
            
#     except:
#         print("!!!!!!")
#         continue
        